In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler



In [3]:
file = '../data/jla_binned_distances_31nodes_v1.txt'

# filefull = '/home/cosmocicatais/Documents/github/neuralCosmoReconstruction/data/jla_lcparams.txt'
syscov = np.loadtxt('../data/cov_jla_binned_distances_31nodes_v1.txt',skiprows=1).reshape((31,31))
# syscov = np.loadtxt('../data/jla_v0_covmatrix.dat',skiprows=1).reshape((740,740))
# data = pd.read_csv(file2, names=['z', '$D_L$'], sep = " ", skiprows=3)
# data = np.loadtxt(file, usecols=[1,4], skiprows=1)
data = np.loadtxt(file, skiprows=3)
nsize = len(syscov)
# print(data.head(31))
# data.head(31) 
# plt.savefig("DistbinnedJLA.png")
np.shape(syscov)

errors = np.diag(syscov).reshape(nsize,1)
errors = np.where(errors!= 0, np.log(errors), 0)
np.shape(errors)
len(data), len(errors)

(31, 31)

In [4]:
shuffle = [0,1,2,3,4,5,6,7,8,30,28,26,24,22,20,9,10,12,14,16,18,29,27,25,23,21,19,17,15,13,11]
data = data[shuffle]

errors= errors[shuffle]
len(data), len(errors), len(shuffle)

(31, 31, 31)

In [5]:
dl = data[:,1]
z = data[:,0]
np.shape(z), np.shape(dl), np.shape(errors), len(data)

((31,), (31,), (31, 1), 31)

In [6]:
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]

zz = np.log(z)
# scalerz = StandardScaler()
# scalerz.fit(z.reshape(-1, 1))
# zz = scalerz.transform(z.reshape((-1,1)))

# scalerdl = StandardScaler()
# scalerdl.fit(dl.reshape(-1, 1))
# dll = scalerdl.transform(dl.reshape((-1,1)))
dll = dl.reshape(nsize, 1)
# scalererr = StandardScaler()
# scalererr.fit(errors.reshape(-1, 1))
# errscl = scalererr.transform(errors.reshape((-1,1)))

z_train, z_test = np.split(zz, indx)
y = np.concatenate((dll, errors), axis=1)
y_train, y_test = np.split(y, indx)

In [7]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([50, 100]))
# HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))
# HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([1, 4, 8]))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([1, 4]))

In [6]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [7]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4, HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [8]:
def hzmodel(input_z, hparams):
    #encoder
    efirst = Dense(hparams[HP_NUM_UNITS1], activation='relu', input_shape=(1,))(input_z)
    x = Dense(hparams[HP_NUM_UNITS2], activation='relu')(efirst)
    x = Dense(hparams[HP_NUM_UNITS3], activation='relu')(x)
#     x = Dense(hparams[HP_NUM_UNITS4], activation='relu')(x)
#     ehidden2 = Dense(hparams[HP_NUM_UNITS3], activation='relu')(ehidden1)
    elast = Dense(2, activation='linear')(x)
        
    return elast

In [9]:
epochs = 200
input_z = tf.keras.layers.Input(shape = (1,))

In [10]:
def train_test_model(hparams):
    model = Model(input_z, hzmodel(input_z, hparams))  
                              
    model.compile(
#             optimizer=hparams[HP_OPTIMIZER],
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_squared_error']
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(X_train, y_train, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks, verbose=0) 
    _, loss = model.evaluate(X_test, y_test)


    return loss
    

In [11]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
#     with tf.summary.FileWriter(run_dir):
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [12]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units1 in HP_NUM_UNITS1.domain.values:
        for num_units2 in HP_NUM_UNITS2.domain.values:
            for num_units3 in HP_NUM_UNITS3.domain.values:
                for num_units4 in HP_NUM_UNITS4.domain.values:
#       for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
#                     for optimizer in HP_OPTIMIZER.domain.values:
                    hparams = {
                      HP_NUM_UNITS1: num_units1,
                      HP_NUM_UNITS2: num_units2,
                      HP_NUM_UNITS3: num_units3,
                      HP_NUM_UNITS4: num_units4,
                    #                       HP_DROPOUT: dropout_rate,
        #                       HP_OPTIMIZER: optimizer,
                      HP_BATCHSIZE: batch_size
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
        #                 print({h.name: hparams[h] for h in hparams})
                    run('logs/hparam_tuning/' + run_name, hparams)
                    session_num += 1

print("Total combinations: {}".format(session_num+1))

--- Starting trial: run-0
1/1 [==============================] - 0s 80ms/step - loss: 450.8255 - mean_squared_error: 450.8255
--- Starting trial: run-1
1/1 [==============================] - 0s 64ms/step - loss: 379.9618 - mean_squared_error: 379.9618
--- Starting trial: run-2
1/1 [==============================] - 0s 64ms/step - loss: 389.6638 - mean_squared_error: 389.6638
--- Starting trial: run-3
1/1 [==============================] - 0s 73ms/step - loss: 310.1361 - mean_squared_error: 310.1361
--- Starting trial: run-4
1/1 [==============================] - 0s 65ms/step - loss: 271.1262 - mean_squared_error: 271.1262
--- Starting trial: run-5
1/1 [==============================] - 0s 71ms/step - loss: 320.1718 - mean_squared_error: 320.1718
--- Starting trial: run-6
1/1 [==============================] - 0s 63ms/step - loss: 318.2217 - mean_squared_error: 318.2217
--- Starting trial: run-7
1/1 [==============================] - 0s 179ms/step - loss: 316.1811 - mean_squared_error: 

In [13]:
# !kill 7439
%tensorboard --logdir logs/hparam_tuning

/bin/bash: line 0: kill: (7439) - No such process
